# core
> Utilities Functions

In [ ]:
#| default_exp core

In [ ]:
#| hide
#| export
from nbdev.showdoc import *
import subprocess
from pathlib import Path
from typing import Union, Optional
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [ ]:
def test_command(command: str) -> Optional[int]:
    """
    Test if a command is available and print its first line of output.
    
    Args:
        command: The command to test (e.g., 'mmseqs', 'samtools', 'blastn').
    
    Returns:
        Optional[int]: Return code from the command, or None if execution fails.
    """
    try:
        result = subprocess.run(
            [command],
            capture_output=True,
            text=True,
            timeout=5
        )
        
        # Print first line of output (usually version/help info)
        if result.stdout:
            first_line = result.stdout.split('\n')[0]
            print(first_line)
        elif result.stderr:
            first_line = result.stderr.split('\n')[0]
            print(first_line)
        
        print(f"Return code: {result.returncode}")
        return result.returncode
        
    except Exception as e:
        print(f"Error running {command}: {e}")
        return None

In [ ]:
test_command('mmseqs')

MMseqs2 (Many against Many sequence searching) is an open-source software suite for very fast, 
Return code: 0


0

In [ ]:
test_command('spades.py')

SPAdes genome assembler v4.2.0
Return code: 0


0

In [ ]:
#| export
def retrieve_and_save_sequence(
    fasta_file: Union[str, Path],
    fasta_id: str,
    start: int,
    end: int,
    output_file: Union[str, Path]
) -> Optional[str]:
    """
    Retrieve a subsequence from a FASTA file and save it to a new file.
    
    Args:
        fasta_file: Path to the input FASTA file.
        fasta_id: The ID of the sequence to retrieve.
        start: The starting position (1-based, inclusive).
        end: The ending position (1-based, inclusive).
        output_file: Path to the output FASTA file.
    
    Returns:
        str: Success message with output file path, or None if error occurs.
    
    Raises:
        ValueError: If fasta_id not found, or if start/end positions are invalid.
        FileNotFoundError: If input FASTA file doesn't exist.
    
    Example:
        >>> retrieve_and_save_sequence('genome.fasta', 'chr1', 100, 200, 'output.fasta')
        'Sequence saved to output.fasta'
    """
    # Convert to Path objects for better path handling
    fasta_file = Path(fasta_file)
    output_file = Path(output_file)
    
    try:
        # Validate input file exists
        if not fasta_file.exists():
            raise FileNotFoundError(f"Input file not found: {fasta_file}")
        
        # Validate coordinates
        if start < 1 or end < 1:
            raise ValueError(f"Coordinates must be >= 1 (got start={start}, end={end})")
        if start > end:
            raise ValueError(f"Start position ({start}) must be <= end position ({end})")
        
        # Search for the sequence
        for record in SeqIO.parse(fasta_file, "fasta"):
            if record.id == fasta_id:
                # Validate coordinates against sequence length
                if end > len(record.seq):
                    raise ValueError(
                        f"End position ({end}) exceeds sequence length ({len(record.seq)})"
                    )
                
                # Extract subsequence (convert to 0-based indexing)
                subsequence = record.seq[start - 1:end]
                
                # Create new record with descriptive ID
                new_record = SeqRecord(
                    subsequence,
                    id=f"{fasta_id}_{start}_{end}",
                    description=f"Extracted from {fasta_id}, positions {start}-{end}"
                )
                
                # Write to output file
                with open(output_file, "w") as output_handle:
                    SeqIO.write(new_record, output_handle, "fasta")
                
                return f"Sequence saved to {output_file}"
        
        # Sequence ID not found
        raise ValueError(f"FASTA ID '{fasta_id}' not found in {fasta_file}")
    
    except Exception as e:
        print(f"Error extracting sequence: {e}")
        return None

In [ ]:
test_fasta = "test.fasta"
seq = 'A C G T A C G T A'.replace(' ','')
#      1 2 3 4 5 6 7 8 9
with open(test_fasta, "w") as f:
    f.write(f">seq1\n{seq}\n")

# Test the function
retrieve_and_save_sequence("test.fasta", "seq1", 3, 7, "output.fasta")

# Check the output
for record in SeqIO.parse("output.fasta", "fasta"):
    print(f"ID: {record.id}")
    print(f"Sequence: {record.seq}")
    assert(record.seq == 'GTACG')
!rm output.fasta

ID: seq1_3_7
Sequence: GTACG


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()